<a href="https://colab.research.google.com/github/nicglaus/prello/blob/Development/algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
from google.colab import auth
import pandas as pd
auth.authenticate_user()

In [82]:
# geo data - link departments & municipalities
query_geo = "SELECT municipality_code, department_code FROM `prello-cna.dbt_chloe_prello.geo_data`"
data_geo = pd.read_gbq(query_geo, project_id="prello-cna")

# Real Estate Sales
query_sales = "SELECT municipality_code, Price_increase FROM `prello-cna.dbt_nicglaus.clean_sales_price_increase`"
data_sales = pd.read_gbq(query_sales, project_id="prello-cna")

# PCT of vacant homes
query_vacant = "SELECT municipality_code, vacant_rate_norm FROM `prello-cna.dbt_nicglaus.clean_vacant_house_2018`"
data_vacant = pd.read_gbq(query_vacant, project_id="prello-cna")

# rent level and housing stress index
query_rent_stress = "SELECT municipality_code, normalised_stress, normalised_rent FROM `prello-cna.dbt_nicglaus.clean_rent_stress`"
data_rent_stress = pd.read_gbq(query_rent_stress, project_id="prello-cna")

# touristic establishments
query_tourist_esta = "SELECT municipality_code, poi_esta_norm FROM `prello-cna.dbt_chloe_prello.clean_tourist_establishments`"
data_tourist_esta = pd.read_gbq(query_tourist_esta, project_id="prello-cna")

# touristic sites
query_tourist_sites = "SELECT municipality_code, poi_sites_norm FROM `prello-cna.dbt_chloe_prello.clean_tourist_sites`"
data_tourist_sites = pd.read_gbq(query_tourist_sites, project_id="prello-cna")

# crime
query_crime = "SELECT department_code, norm_2021_crime_rate FROM `prello-cna.dbt_nicglaus.clean_crime`"
data_crime = pd.read_gbq(query_crime, project_id="prello-cna")

# GDP per capita
query_gdp = "SELECT department_code, pib_growth_norm, pib_2020_norm FROM `prello-cna.dbt_nicglaus.clean_pib_capita`"
data_gdp = pd.read_gbq(query_gdp, project_id="prello-cna")

# unemployment
query_unemployment = "SELECT department_code, unemployment_norm FROM `prello-cna.dbt_nicglaus.clean_unemployment_rate`"
data_unemployment = pd.read_gbq(query_unemployment, project_id="prello-cna")

# population growth
query_population = "SELECT municipality_code, popu_growth FROM `prello-cna.dbt_chloe_prello.clean_pop_growth`"
data_population = pd.read_gbq(query_population, project_id="prello-cna")



In [83]:
# Merge the DataFrames based on a common key (city_id)
merged_data = pd.merge(data_geo, data_sales, on='municipality_code', how='inner')
merged_data = pd.merge(merged_data, data_population, on='municipality_code', how='inner')
merged_data = pd.merge(merged_data, data_vacant, on='municipality_code', how='inner')
merged_data = pd.merge(merged_data, data_rent_stress, on='municipality_code', how='inner')
merged_data = pd.merge(merged_data, data_tourist_esta, on='municipality_code', how='inner')
merged_data = pd.merge(merged_data, data_tourist_sites, on='municipality_code', how='inner')
merged_data = pd.merge(merged_data, data_crime, on='department_code', how='inner')
merged_data = pd.merge(merged_data, data_gdp, on='department_code', how='inner')
merged_df = pd.merge(merged_data, data_unemployment, on='department_code', how='inner')

In [84]:
merged_df
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4158 entries, 0 to 4157
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   municipality_code     4158 non-null   object 
 1   department_code       4158 non-null   object 
 2   Price_increase        4158 non-null   float64
 3   popu_growth           4158 non-null   float64
 4   vacant_rate_norm      4158 non-null   float64
 5   normalised_stress     4158 non-null   float64
 6   normalised_rent       4158 non-null   float64
 7   poi_esta_norm         4158 non-null   float64
 8   poi_sites_norm        4158 non-null   float64
 9   norm_2021_crime_rate  4158 non-null   float64
 10  pib_growth_norm       4158 non-null   float64
 11  pib_2020_norm         4158 non-null   float64
 12  unemployment_norm     4158 non-null   float64
dtypes: float64(11), object(2)
memory usage: 454.8+ KB


In [85]:
 inv_div_weights = {'Price_increase': 0.11,
 'vacant_rate_norm': 0.16,
 'normalised_stress': 0.05,
 'normalised_rent': 0.05,
 'poi_esta_norm': 0.11,
 'poi_sites_norm': 0.10,
 'norm_2021_crime_rate': 0.07,
 'pib_growth_norm': 0.09,
 'unemployment_norm': 0.09,
 'popu_growth': 0.05}

In [86]:
# Apply the weighting to each feature and create a new 'weighted_total' column
for feature, weight in inv_div_weights.items():
    merged_df[feature] = merged_df[feature] * weight

merged_df['weighted_total'] = merged_df[['Price_increase', 'popu_growth', 'vacant_rate_norm', 'normalised_stress', 'normalised_rent', 'poi_esta_norm', 'poi_sites_norm', 'norm_2021_crime_rate', 'pib_growth_norm', 'pib_2020_norm', 'unemployment_norm']].sum(axis=1)
merged_df.head()


,municipality_code,department_code,Price_increase,popu_growth,vacant_rate_norm,normalised_stress,normalised_rent,poi_esta_norm,poi_sites_norm,norm_2021_crime_rate,pib_growth_norm,pib_2020_norm,unemployment_norm,weighted_total
0,10099,10,0.106619,0.049335,0.142508,0.005556,0.006994,0.000000,0.0,0.055602,0.0,0.061132,0.027237,0.454982
1,10030,10,0.143836,0.063380,0.152127,0.005556,0.008733,0.000329,0.0,0.055602,0.0,0.061132,0.027237,0.517931
2,10060,10,0.132363,0.063103,0.144785,0.005556,0.010560,0.000000,0.0,0.055602,0.0,0.061132,0.027237,0.500338
3,10081,10,0.122938,0.043605,0.151176,0.005556,0.007811,0.000164,0.0,0.055602,0.0,0.061132,0.027237,0.475220
4,10238,10,0.125145,0.055181,0.145128,0.005556,0.004628,0.000246,0.0,0.055602,0.0,0.061132,0.027237,0.479855


In [87]:
merged_df.to_csv('test.csv')